In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [4]:
!pip install git+https://github.com/ultralytics/ultralytics.git@main

  Cloning https://github.com/ultralytics/ultralytics.git (to revision main) to /tmp/pip-req-build-whwf2d3u
  Running command git clone --filter=blob:none --quiet https://github.com/ultralytics/ultralytics.git /tmp/pip-req-build-whwf2d3u
  Resolved https://github.com/ultralytics/ultralytics.git to commit 3d4a86d2ee35e52b99a65f35126f758078fcccac
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.3.66-py3-none-any.whl size=911405 sha256=ccd5ef41c2a78c6e84dda69cd969d41a1f9e77028d54cfac0ef70746497ee7bd
  Stored in directory: /tmp/pip-ephem-wheel-cache-qoucrzcb/wheels/ce/28/0b/747d53a7fba3579e3367c6a4b40abf971babc68f7a46b16a28
Successfully built ultralytics


In [ ]:
#Run inference on images

from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('yolov8n.pt')

# Specify the classes to detect (indices for 'car', 'motorcycle', 'bus', 'truck')
classes_to_detect = [2, 3, 5, 7]

# Define path to directory containing images and videos for inference
source = '/content/drive/My Drive/Colab Notebooks/images_for_inference'

# Run inference on the source
results = model.predict(source, stream=True, classes=classes_to_detect)  # generator of Results objects

for index, result in enumerate(results):
    boxes = result.boxes  # Boxes object for bounding box outputs
    probs = result.probs  # Probs object for classification outputs
    result.show()  # display to screen
    filename = f'/content/drive/My Drive/Colab Notebooks/images_for_inference_output/result_{index}.jpg'
    result.save(filename=filename)

In [ ]:
# inference on Youtube videos

import random
import cv2
import numpy as np
from ultralytics import YOLO
from datetime import datetime
from IPython.display import HTML
from base64 import b64encode
from tqdm.notebook import tqdm

# Opening the file in read mode, reading the classes, and generating random colors
my_file = open("/content/drive/My Drive/Colab Notebooks/coco.txt", "r")
data = my_file.read()
class_list = data.split("\n")
my_file.close()
detection_colors = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for _ in class_list]

# Load a pretrained YOLO model
model = YOLO("weights/yolov8n.pt", "v8")  # Ensure you have the correct path

# Specify classes to detect (indices for 'car', 'motorcycle', 'bus', 'truck')
classes_to_detect = [2, 3, 5, 7]

# Video setup
cap = cv2.VideoCapture("/content/drive/My Drive/Colab Notebooks/videos_for_inference/afriq1.MP4")
if not cap.isOpened():
    print("Cannot open camera")
    exit()
frame_wid = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_hyt = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Output video setup
timestamp = datetime.now().strftime("%d-%m-%Y _ %H.%M.%S")
output_filename = f"/content/drive/My Drive/Colab Notebooks/videos_for_inference_output/output_{timestamp}.mp4"
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter(output_filename, fourcc, fps, (frame_wid, frame_hyt))

# Object detection loop
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
for _ in tqdm(range(total_frames), desc="Processing Video"):
    ret, frame = cap.read()
    if not ret:
        break
    detect_params = model.predict(source=[frame], conf=0.45, save=False, classes=classes_to_detect)
    detect_params_tensor = detect_params[0].boxes.data
    DP = detect_params_tensor.numpy()
    if len(DP) != 0:
        for i in range(len(detect_params[0].boxes)):
            box = detect_params[0].boxes[i]
            clsID, conf, bb = box.cls.numpy()[0], box.conf.numpy()[0], box.xyxy.numpy()[0]
            cv2.rectangle(frame, (int(bb[0]), int(bb[1])), (int(bb[2]), int(bb[3])), detection_colors[int(clsID)], 3)
            cv2.putText(frame, f"{class_list[int(clsID)]} {round(conf, 3)}%", (int(bb[0]), int(bb[1]) - 10), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
    out.write(frame)

# Cleanup
cap.release()
out.release()

# Function to display video
def display_video(video_path):
    video_file = open(video_path, "rb")
    video_data = video_file.read()
    video_encoded = b64encode(video_data).decode("ascii")
    video_file.close()
    video_html = HTML(f"""
    <video width="640" height="480" controls>
        <source src="data:video/mp4;base64,{video_encoded}" type="video/mp4">
    </video>
    """)
    display(video_html)

# Display the processed video
display_video(output_filename)